In [146]:
import pandas as pd
import numpy as np
import itertools
import random
from dataclasses import dataclass

In [161]:
@dataclass
class GameResult:
    player_hand: tuple
    player_hand_value: int
    banker_hand: tuple
    banker_hand_value: int
    last_action: str
    winner: str

In [148]:
@dataclass
class BaccaratResult:
    game: str
    game_id: str
    player_id: str
    status: str
    # start_time: datetime
    # end_time: datetime
    player_hand: list
    player_hand_value: int
    banker_hand: list
    banker_hand_value: int
    player_beginning_balance: float
    player_wager: float
    player_bet: str
    player_payout: float
    player_ending_balance: float
    last_action: str
    game_outcome: str

In [149]:
def build_deck():

    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    original_deck = [f"{rank} of {suit}" for suit in suits for rank in ranks]
    
    return original_deck

In [150]:
def build_combinations(deck):

    card_combinations = list(itertools.combinations(deck, 2))
    card_combinations_df = pd.DataFrame(card_combinations, columns = ['card_1', 'card_2'])

    card_combinations_df['card_combination'] = card_combinations_df.apply(
        lambda row:
            (row['card_1'], row['card_2']), 
            axis = 1
    )

    card_combinations_df['card_1_value'] = card_combinations_df['card_1'].apply(
        lambda x: 
            int(x.split(' ')[0]) if x.split(' ')[0].isdigit() else 1 if x.split(' ')[0] == 'A' else 10
    )

    card_combinations_df['card_2_value'] = card_combinations_df['card_2'].apply(
        lambda x: 
            int(x.split(' ')[0]) if x.split(' ')[0].isdigit() else 1 if x.split(' ')[0] == 'A' else 10
    )

    card_combinations_df['card_combination_value'] = card_combinations_df.apply(
    lambda row:
        int(str(row['card_1_value'] + row['card_2_value'])[-1]),
        axis = 1
    )

    deck_dict = dict(zip(card_combinations_df['card_combination'], card_combinations_df['card_combination_value']))

    return card_combinations, card_combinations_df, deck_dict

In [151]:
def draw_player_hand():
    
    deck = build_deck()
    card_combinations, card_combinations_df, deck_dict = build_combinations(deck)

    player_hand = tuple(random.choice(card_combinations))
    player_hand_value = int(str(deck_dict[player_hand])[-1])

    return player_hand, player_hand_value

In [152]:
def rebuild_deck(original_deck, excluded_cards: list):
    
    deck_remaining = [card for card in original_deck if card not in excluded_cards]

    return deck_remaining

In [153]:
def draw_banker_hand(player_hand, with_weights = 'No'):

    original_deck = build_deck()
    deck_remaining = rebuild_deck(original_deck, player_hand)
    card_combinations, card_combinations_df, deck_dict = build_combinations(deck_remaining)

    if with_weights != 'No':
        weights = np.where(card_combinations_df['card_combination_value'] >= 7, 3, 1)

    else:
        weights = np.ones(len(card_combinations))

    
    banker_hand = tuple(random.choices(card_combinations, weights = weights)[0])
    banker_hand_value = int(str(deck_dict[banker_hand])[-1])

    return banker_hand, banker_hand_value

In [154]:
def draw_player(drawn_cards, player_hand, player_hand_value):
    
    original_deck = build_deck()
    deck_remaining = rebuild_deck(original_deck, drawn_cards)
    card_combinations, card_combinations_df, deck_dict = build_combinations(deck_remaining)

    player_draw = (random.choice(deck_remaining), )
    player_draw_rank = player_draw[0].split(' ')[0]
    player_draw_value = int(player_draw_rank) if player_draw_rank.isdigit() else 1 if player_draw_rank == 'A' else 10
    player_hand = player_hand + player_draw    
    player_hand_value =  int(str(player_hand_value + player_draw_value)[-1])

    return player_hand, player_hand_value, player_draw

In [155]:
def draw_banker(drawn_cards: list, banker_hand, banker_hand_value):
    
    original_deck = build_deck()
    deck_remaining = rebuild_deck(original_deck, drawn_cards)
    card_combinations, card_combinations_df, deck_dict = build_combinations(deck_remaining)

    banker_draw = (random.choice(deck_remaining), )
    banker_draw_rank = banker_draw[0].split(' ')[0]
    banker_draw_value = int(banker_draw_rank) if banker_draw_rank.isdigit() else 1 if banker_draw_rank == 'A' else 10
    banker_hand = banker_hand + banker_draw    
    banker_hand_value = int(str(banker_hand_value + banker_draw_value)[-1])

    return banker_hand, banker_hand_value, banker_draw

In [156]:
def announce_winner(player_hand_value, banker_hand_value):
    winner = str()

    if player_hand_value > banker_hand_value:
        winner = 'Player'

    elif player_hand_value < banker_hand_value:
        winner = 'Banker'

    else:
        winner = 'Tie'

    return winner

In [157]:
def compute_payout(winner, wager, bet):
    if winner == 'Player':
        if bet == 'Player':
            payout = wager + (wager * 1)
        else:
            payout = 0

    elif winner == 'Banker':
        if bet == 'Banker':
            payout = wager + (wager * .95)
        else:
            payout = 0

    elif winner == 'Tie':
        if bet == 'Tie':
            payout = wager + (wager * 8)
        else:
            payout = 0

    return payout

In [158]:
def play_game(type = 'normal'):

    # 1. Draw Player Cards
    player_hand, player_hand_value = draw_player_hand()
    drawn_cards = player_hand
    
    # 2. Draw Banker Cards
    if type == 'normal':
        banker_hand, banker_hand_value = draw_banker_hand(player_hand, 'No')
        drawn_cards += tuple(banker_hand)
    
    elif type == 'rigged':
        banker_hand, banker_hand_value = draw_banker_hand(player_hand, with_weights='Yes')
        drawn_cards += tuple(banker_hand)

    # 3. Decision logic for drawing additional card for both player and banker
    if player_hand_value <= 5:
        
        player_hand, player_hand_value, player_draw = draw_player(drawn_cards, player_hand, player_hand_value)
        drawn_cards += tuple(player_draw)
        last_action = 'player_draw'

    else:
        last_action = 'initial_deal'

    
    if banker_hand_value <= 5:
        
        banker_hand, banker_hand_value, banker_draw = draw_banker(drawn_cards, banker_hand, banker_hand_value)
        drawn_cards += tuple(banker_draw)
        last_action = 'banker_draw'

    else:
        last_action = 'initial_deal'

    winner = announce_winner(player_hand_value, banker_hand_value)

    
    return GameResult(player_hand, player_hand_value, banker_hand, banker_hand_value, last_action, winner)

In [ ]:
# Simulation

player_score = 0
banker_score = 0
tie = 0
games = 10000

for i in range(games):
    player_hand, player_hand_value, banker_hand, banker_hand_value, last_action = play_game('rigged')
    if player_hand_value > banker_hand_value:
        player_score+=1
    elif player_hand_value < banker_hand_value:
        banker_score+=1
    elif player_hand_value == banker_hand_value:
        tie+=1

    assert len(set(player_hand).intersection(set(banker_hand))) == 0
    

print(f'Player Score: {player_score} | Win Rate: {player_score/games}')
print(f'Banker Score: {banker_score} | Win Rate: {banker_score/games}')
print(f'Tie: {tie}')

Player Score: 3610 | Win Rate: 0.361
Banker Score: 5151 | Win Rate: 0.5151
Tie: 1239


In [176]:
result = play_game('normal')
result.banker_hand, result.banker_hand_value

(('9 of Clubs', '9 of Spades'), 8)

In [171]:
result[0][0].split(' of ')[1]

TypeError: 'GameResult' object is not subscriptable

In [116]:
player_hand = [
    {
        'value': result[0][0].split(' of ')[1],
        'rank': result[0][0].split(' of ')[0]
    },
    {
        'value': result[0][1].split(' of ')[1],
        'rank': result[0][1].split(' of ')[0]
    },
]


In [117]:
player_hand

[{'value': 'Hearts', 'rank': '8'}, {'value': 'Diamonds', 'rank': '3'}]

In [118]:
if len(result[0]) == 3:
    player_hand.append(
        {
            'value': result[0][2].split(' of ')[1],
            'rank': result[0][2].split(' of ')[0]
        }
    )

In [119]:
len(result[0]) == 3

True

In [120]:
player_hand

[{'value': 'Hearts', 'rank': '8'},
 {'value': 'Diamonds', 'rank': '3'},
 {'value': 'Clubs', 'rank': 'A'}]